# Data classification based on categories and sub categories

In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [2]:
df = pd.read_excel("Data.xlsx")

In [3]:
dfclassification = pd.read_excel("Classification details.xlsx")

In [4]:
dfclassification

,Unnamed: 0,Code secteur,Nom Secteur
0,0,1.0,1 PRESTATIONS TRANSPORT & LOGISTIQUE
1,1,1.1,1.1 SERVICES TRANSPORT
2,2,NaN,1.1.1 TRANSPORTS ROUTIERS GÉNÉRAL CARGO
3,3,NaN,1111 - Transport routier régional et national ...
4,4,NaN,1112 - Transport de vrac solide
...,...,...,...
169,169,NaN,3224 - Location de matériel de stockage
170,170,NaN,3.2.4 EMBALLAGE
171,171,NaN,3241 - Fabricant de matériel d’emballage
172,172,NaN,"3242 - Calages, films, feuillards"


In [5]:
dfclassification = dfclassification.loc[:, ~dfclassification.columns.str.contains('^Unnamed')]

In [6]:
writer = pd.ExcelWriter('Classification details.xlsx')
dfclassification.to_excel(writer)
writer.save()

# Preparing sub categories

In [7]:
dfSubCategories = DataFrame(columns=['Code Secteur','Nom Secteur'])

In [8]:
Code_Secteur = []
Secteur = []

for i in range(174):
    if dfclassification['Nom Secteur'][i][0:5].find('.') == -1:
        Code_Secteur.append(dfclassification['Nom Secteur'][i][0:5])
        Secteur.append(dfclassification['Nom Secteur'][i][5:])

In [9]:
len(Secteur)

126

In [10]:
len(Code_Secteur)

126

In [11]:
dfSubCategories['Code Secteur'] = Code_Secteur
dfSubCategories['Nom Secteur'] = Secteur

In [12]:
dfSubCategories.head()

,Code Secteur,Nom Secteur
0,1 P,RESTATIONS TRANSPORT & LOGISTIQUE
1,1111,- Transport routier régional et national par lots
2,1112,- Transport de vrac solide
3,1113,- Groupage et distribution
4,1122,- Transport de meubles


In [13]:
dfSubCategories = dfSubCategories.iloc[1:]

In [14]:
dfSubCategories.head()

,Code Secteur,Nom Secteur
1,1111,- Transport routier régional et national par lots
2,1112,- Transport de vrac solide
3,1113,- Groupage et distribution
4,1122,- Transport de meubles
5,1123,- Transport de produits d’hygiène-santé


In [15]:
new_index = np.arange(125)

In [16]:
dfSubCategories = dfSubCategories.reindex(new_index, fill_value=' ')

In [17]:
dfSubCategories.head()

,Code Secteur,Nom Secteur
0,,
1,1111,- Transport routier régional et national par lots
2,1112,- Transport de vrac solide
3,1113,- Groupage et distribution
4,1122,- Transport de meubles


# Preparing categories

In [18]:
dfCategories = DataFrame(columns=['Code Secteur','Nom Secteur'])

In [19]:
Code_Secteur = []
Secteur = []

for i in range(174):
    if dfclassification['Nom Secteur'][i][0:5].find('.') != -1:
        if dfclassification['Nom Secteur'][i][0:5][3] == '.':
            Code_Secteur.append(dfclassification['Nom Secteur'][i][0:5])
            Secteur.append(dfclassification['Nom Secteur'][i][5:])

In [20]:
dfCategories['Code Secteur'] = Code_Secteur
dfCategories['Nom Secteur'] = Secteur

In [37]:
dfCategories.head()

,Code Secteur,Nom Secteur
0,1.1.1,TRANSPORTS ROUTIERS GÉNÉRAL CARGO
1,1.1.2,TRANSPORTS SPÉCIALISÉS
2,1.1.3,TRANSPORTS EN CITERNE
3,1.1.4,MESSAGERIE
4,1.1.5,TRANSPORT EXPRESS OU URGENT


In [38]:
len(dfCategories['Nom Secteur'])

37

# Level 1 : sub categories

In [32]:
dfsub = df.copy()

In [33]:
dfsub = dfsub.loc[:, ~dfsub.columns.str.contains('^Unnamed')]

In [61]:
listData = []
for i in range(1589):
    listData.append(''.join(str([dfsub['Nom'][i],dfsub['Description'][i],dfsub['Secteur'][i],dfsub['Activite'][i]])))

In [29]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [3]:
dfSubCatClass = DataFrame(columns=['Nom','Code Secteur','Nom Secteur'])
listNom = []
listCode = []
listNomSecteur = []
k=0
for item in listData:
    listNom.append(dfsub['Nom'][k])
    dictData = {}
    for j in range(125):
        dictData[dfSubCategories['Nom Secteur'][j]] = cosine_sim(item,dfSubCategories['Nom Secteur'][j])
        
    listCode.append( max(dictData.values()) )
    listNomSecteur.append( list(dictData.keys())[list(dictData.values()).index(max(dictData.values()))] )
    
    
    k = k+1
    print("\n")

In [36]:
len(listCode)

1589

In [37]:
len(listNomSecteur)

1589

In [39]:
dfSubCatClass['Nom'] = listNom
dfSubCatClass['Code Secteur'] = listCode
dfSubCatClass['Nom Secteur'] = listNomSecteur

In [45]:
dfSubCatClass

,Nom,Code Secteur,Nom Secteur
,,,
0,Allo Fret,0.563834,- Transport de meubles
1,DHL Express,0.235836,- Transport et livraison
2,AAC Globe Express,0.618945,- Transport de meubles
3,France Express,0.499754,- Transport express national
4,DHL Express,0.283636,- Transport et livraison
...,...,...,...
1584,Awell Ttl,0.161714,- Logistique de retour
1585,Axalys Logistics Maroc,0.150557,- Transport de meubles
1586,Aya Intercontinental,0.161714,- Transport de meubles


In [49]:
# dfSubCategories

dfSubCatClass.rename(columns = {"Code Secteur" : "Similarity"}, inplace = True)

In [55]:
dfSubCatClass

,Nom,Similarity,Nom Secteur
,,,
0,Allo Fret,0.563834,- Transport de meubles
1,DHL Express,0.235836,- Transport et livraison
2,AAC Globe Express,0.618945,- Transport de meubles
3,France Express,0.499754,- Transport express national
4,DHL Express,0.283636,- Transport et livraison
...,...,...,...
1584,Awell Ttl,0.161714,- Logistique de retour
1585,Axalys Logistics Maroc,0.150557,- Transport de meubles
1586,Aya Intercontinental,0.161714,- Transport de meubles


In [24]:
dfSubCatClass['Nom Secteur']

0             - Transport de meubles
1           - Transport et livraison
2             - Transport de meubles
3       - Transport express national
4           - Transport et livraison
                    ...             
1584          - Logistique de retour
1585          - Transport de meubles
1586          - Transport de meubles
1587          - Logistique de retour
1588          - Logistique de retour
Name: Nom Secteur, Length: 1589, dtype: object

In [26]:
listCode = []
listSecteur = []
for i in range(1589):
    for j in range(125):
        if  dfSubCatClass['Nom Secteur'][i] == dfSubCategories['Nom Secteur'][j]:
            listCode.append(dfSubCategories['Code Secteur'][j])
            listSecteur.append(dfSubCategories['Nom Secteur'][j])
            break

In [78]:
len(listCode)

1589

In [80]:
dfSubCatClass['Code Secteur'] = listCode

In [81]:
dfSubCatClass

,Nom,Similarity,Nom Secteur,Code Secteur
,,,,
0,Allo Fret,0.563834,- Transport de meubles,1122
1,DHL Express,0.235836,- Transport et livraison,1231
2,AAC Globe Express,0.618945,- Transport de meubles,1122
3,France Express,0.499754,- Transport express national,1151
4,DHL Express,0.283636,- Transport et livraison,1231
...,...,...,...,...
1584,Awell Ttl,0.161714,- Logistique de retour,1234
1585,Axalys Logistics Maroc,0.150557,- Transport de meubles,1122
1586,Aya Intercontinental,0.161714,- Transport de meubles,1122


In [82]:
writer = pd.ExcelWriter('Classification sub categories.xlsx')
dfSubCatClass.to_excel(writer)
writer.save()

In [22]:
dfSubCatClass = pd.read_excel("Classification sub categories.xlsx")

In [23]:
dfSubCatClass

,Unnamed: 0,Nom,Similarity,Nom Secteur,Code Secteur
0,0,Allo Fret,0.563834,- Transport de meubles,1122
1,1,DHL Express,0.235836,- Transport et livraison,1231
2,2,AAC Globe Express,0.618945,- Transport de meubles,1122
3,3,France Express,0.499754,- Transport express national,1151
4,4,DHL Express,0.283636,- Transport et livraison,1231
...,...,...,...,...,...
1584,1584,Awell Ttl,0.161714,- Logistique de retour,1234
1585,1585,Axalys Logistics Maroc,0.150557,- Transport de meubles,1122
1586,1586,Aya Intercontinental,0.161714,- Transport de meubles,1122
1587,1587,Ayan Transit,0.161714,- Logistique de retour,1234


# Level 2 : categories

In [4]:
dfCatClass = DataFrame(columns=['Nom','Code Secteur','Nom Secteur'])
listNom = []
listCode = []
listNomSecteur = []
k=0
for item in listData:
    listNom.append(dfsub['Nom'][k])
    dictData = {}
    for j in range(37):
        dictData[dfCategories['Nom Secteur'][j]] = cosine_sim(item,dfCategories['Nom Secteur'][j])
        
    listCode.append( max(dictData.values()) )
    listNomSecteur.append( list(dictData.keys())[list(dictData.values()).index(max(dictData.values()))] )
    
    
    k = k+1

In [43]:
dfCatClass['Nom'] = listNom
dfCatClass['Code Secteur'] = listCode
dfCatClass['Nom Secteur'] = listNomSecteur

In [44]:
dfCatClass

,Nom,Code Secteur,Nom Secteur
0,Allo Fret,0.516752,TRANSPORT ROUTIER INTERNATIONAL
1,DHL Express,0.178742,CONSEIL (EN LOGISTIQUE) - FORMATION ET EMPLOI
2,AAC Globe Express,0.551207,TRANSPORT ROUTIER INTERNATIONAL
3,France Express,0.400367,TRANSPORT EXPRESS OU URGENT
4,DHL Express,0.200818,TRANSPORTS EN CITERNE
...,...,...,...
1584,Awell Ttl,0.161714,LOGISTIQUE AVANCÉE
1585,Axalys Logistics Maroc,0.150557,TRANSPORTS SPÉCIALISÉS
1586,Aya Intercontinental,0.410363,TRANSPORT INTERCONTINENTAL
1587,Ayan Transit,0.161714,LOGISTIQUE AVANCÉE


In [45]:
dfCatClass.rename(columns = {"Code Secteur" : "Similarity"}, inplace = True)

In [46]:
listCode = []
listSecteur = []
for i in range(1589):
    for j in range(37):
        if  dfCatClass['Nom Secteur'][i] == dfCategories['Nom Secteur'][j]:
            listCode.append(dfCategories['Code Secteur'][j])
            listSecteur.append(dfCategories['Nom Secteur'][j])
            break

In [47]:
dfCatClass['Code Secteur'] = listCode

In [48]:
dfCatClass

,Nom,Similarity,Nom Secteur,Code Secteur
0,Allo Fret,0.516752,TRANSPORT ROUTIER INTERNATIONAL,1.1.7
1,DHL Express,0.178742,CONSEIL (EN LOGISTIQUE) - FORMATION ET EMPLOI,1.5.2
2,AAC Globe Express,0.551207,TRANSPORT ROUTIER INTERNATIONAL,1.1.7
3,France Express,0.400367,TRANSPORT EXPRESS OU URGENT,1.1.5
4,DHL Express,0.200818,TRANSPORTS EN CITERNE,1.1.3
...,...,...,...,...
1584,Awell Ttl,0.161714,LOGISTIQUE AVANCÉE,1.2.3
1585,Axalys Logistics Maroc,0.150557,TRANSPORTS SPÉCIALISÉS,1.1.2
1586,Aya Intercontinental,0.410363,TRANSPORT INTERCONTINENTAL,1.1.8
1587,Ayan Transit,0.161714,LOGISTIQUE AVANCÉE,1.2.3


# Level 3 : Maximum similarity between catg and sub catg

In [49]:
dfCatClass.rename(columns = {"Code Secteur" : "Categorie code"}, inplace = True)
dfCatClass.rename(columns = {"Similarity" : "Categorie Similarity"}, inplace = True)
dfCatClass.rename(columns = {"Nom Secteur" : "Categorie"}, inplace = True)

In [51]:
dfCatClass.head()

,Nom,Categorie Similarity,Categorie,Categorie code
0,Allo Fret,0.516752,TRANSPORT ROUTIER INTERNATIONAL,1.1.7
1,DHL Express,0.178742,CONSEIL (EN LOGISTIQUE) - FORMATION ET EMPLOI,1.5.2
2,AAC Globe Express,0.551207,TRANSPORT ROUTIER INTERNATIONAL,1.1.7
3,France Express,0.400367,TRANSPORT EXPRESS OU URGENT,1.1.5
4,DHL Express,0.200818,TRANSPORTS EN CITERNE,1.1.3


In [53]:
dfCatClass['Sub Categorie Similarity'] = dfSubCatClass['Similarity']
dfCatClass['Sub Categorie'] = dfSubCatClass['Nom Secteur']
dfCatClass['Sub Categorie code'] = dfSubCatClass['Code Secteur']

In [54]:
dfCatClass.head()

,Nom,Categorie Similarity,Categorie,Categorie code,Sub Categorie Similarity,Sub Categorie,Sub Categorie code
0,Allo Fret,0.516752,TRANSPORT ROUTIER INTERNATIONAL,1.1.7,0.563834,- Transport de meubles,1122
1,DHL Express,0.178742,CONSEIL (EN LOGISTIQUE) - FORMATION ET EMPLOI,1.5.2,0.235836,- Transport et livraison,1231
2,AAC Globe Express,0.551207,TRANSPORT ROUTIER INTERNATIONAL,1.1.7,0.618945,- Transport de meubles,1122
3,France Express,0.400367,TRANSPORT EXPRESS OU URGENT,1.1.5,0.499754,- Transport express national,1151
4,DHL Express,0.200818,TRANSPORTS EN CITERNE,1.1.3,0.283636,- Transport et livraison,1231


In [55]:
writer = pd.ExcelWriter('Total DataClassification .xlsx')
dfCatClass.to_excel(writer)
writer.save()

In [60]:
dfFinalClassif = DataFrame(columns = ['Nom','Secteur','Code Secteur'])

In [66]:
# Take categorie and code categorie from the classification with max similarity

listSecteur = []
listCodeSecteur = []

for i in range(1589):
    maxSimilarity = max(dfCatClass['Categorie Similarity'][i], dfCatClass['Sub Categorie Similarity'][i])
    if maxSimilarity == dfCatClass['Categorie Similarity'][i]:
        listSecteur.append(dfCatClass['Categorie'][i])
        listCodeSecteur.append(dfCatClass['Categorie code'][i])
    elif maxSimilarity == dfCatClass['Sub Categorie Similarity'][i]:
        listSecteur.append(dfCatClass['Sub Categorie'][i])
        listCodeSecteur.append(dfCatClass['Sub Categorie code'][i])
    else:
        listSecteur.append("Prestations transport et logistique")
        listCodeSecteur.append("1")

In [74]:
for i in range(1589):
    if listSecteur[i].find('-') != -1:
        listSecteur[i] = listSecteur[i][1:]
        

In [76]:
dfFinalClassif['Nom'] = dfCatClass['Nom']
dfFinalClassif['Secteur'] = listSecteur
dfFinalClassif['Code Secteur'] = listCodeSecteur

In [78]:
writer = pd.ExcelWriter('Data Classification.xlsx')
dfFinalClassif.to_excel(writer)
writer.save()